# CSE251B Project Milestone Starter File

## Step 1: Import Dependencies:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
import tqdm

## Step 2: Load the Dataset

#### You need to describe in your own words what the dataset is about, and use mathematical language and formulate your prediction task on the submitted PDF file for Question 1 Problem A.

#### Here we are loading the dataset from the local directory. And answer Question 1 Problem B

In [2]:
train_npz = np.load('cse-251-b-2025/train.npz')
train_data = train_npz['data']
test_npz  = np.load('cse-251-b-2025/test_input.npz')
test_data  = test_npz['data']

In [3]:
X_train = train_data[..., :50, :]

aug_X = []
max_swaps_per_scene = 8  #
for scene in train_data:
    vehicle_indices = []

    # Find vehicle agents (object_type == 0)
    for i in range(1, 50):  # skip agent 0
        agent = scene[i]
        if np.all(agent == 0):
            continue
        if agent[0, 5] == 0:  # 'vehicle'
            vehicle_indices.append(i)

    # Randomly select up to max_swaps_per_scene
    selected = np.random.choice(vehicle_indices, size=min(max_swaps_per_scene, len(vehicle_indices)), replace=False)

    for i in selected:
        swapped_scene = scene.copy()
        swapped_scene[[0, i]] = swapped_scene[[i, 0]]
        aug_X.append(swapped_scene[:, :50, :])  # (50, 50, 6)

# Stack and combine
aug_X = np.stack(aug_X)                          # (N_aug, 50, 50, 6)
X_train_aug = np.concatenate([X_train, aug_X], axis=0)

print("Original X_train shape:", X_train.shape)
print("Augmented X_train shape:", X_train_aug.shape)

X_train = X_train_aug
Y_train = train_data[:, 0, 50:, :2]

Original X_train shape: (10000, 50, 50, 6)
Augmented X_train shape: (89784, 50, 50, 6)


In [4]:
def plot_heatmap(data, title=None, bins=5):
    plt.figure(figsize=(6, 6))

    x_max = data[..., 0].max()
    x_min = data[..., 0].min()
    y_max = data[..., 1].max()
    y_min = data[..., 1].min()

    plt.hist2d(data[:, 0], data[:, 1], bins=bins, cmap='hot')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.title(title)
    plt.colorbar(label='Density')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.show()

In [5]:
xy_in = train_data[:, :, :50, :2].reshape(-1, 2)
# only find the x, y != 0
xy_in_not_0 = xy_in[(xy_in[:, 0] != 0) & (xy_in[:, 1] != 0)]

In [8]:
plot_heatmap(xy_in, title='Heatmap of XY In', bins=5)
plot_heatmap(xy_in_not_0, title='Heatmap of XY In (non-zero)', bins=5)

NameError: name 'plot_heatmap' is not defined

In [ ]:
plot_heatmap(xy_in, title='Heatmap of XY In', bins=50)
plot_heatmap(xy_in_not_0, title='Heatmap of XY In (non-zero)', bins=50)

#### Try to play around with dataset for training and testing, make exploratory analysis on the dataset for bonus points(up to 2)

## Step 3: Setting up the Training and Testing

### Example Code:

In [4]:
class TrajectoryDatasetTrain(Dataset):
    def __init__(self, data, scale=10.0, augment=True):
        """
        data: Shape (N, 50, 110, 6) Training data
        scale: Scale for normalization (suggested to use 10.0 for Argoverse 2 data)
        augment: Whether to apply data augmentation (only for training)
        """
        self.data = data
        self.scale = scale
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        scene = self.data[idx]
        # Getting 50 historical timestamps and 60 future timestamps
        hist = scene[:, :50, :].copy()    # (agents=50, time_seq=50, 6)
        future = torch.tensor(scene[0, 50:, :2].copy(), dtype=torch.float32)  # (60, 2)
        
        # Data augmentation(only for training)
        if self.augment:
            if np.random.rand() < 0.5:
                theta = np.random.uniform(-np.pi, np.pi)
                R = np.array([[np.cos(theta), -np.sin(theta)],
                              [np.sin(theta),  np.cos(theta)]], dtype=np.float32)
                # Rotate the historical trajectory and future trajectory
                hist[..., :2] = hist[..., :2] @ R
                hist[..., 2:4] = hist[..., 2:4] @ R
                future = future @ R
            if np.random.rand() < 0.5:
                hist[..., 0] *= -1
                hist[..., 2] *= -1
                future[:, 0] *= -1

        # Use the last timeframe of the historical trajectory as the origin
        origin = hist[0, 49, :2].copy()  # (2,)
        hist[..., :2] = hist[..., :2] - origin
        future = future - origin

        # Normalize the historical trajectory and future trajectory
        hist[..., :4] = hist[..., :4] / self.scale
        future = future / self.scale

        data_item = Data(
            x=torch.tensor(hist, dtype=torch.float32),
            y=future.type(torch.float32),
            origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
            scale=torch.tensor(self.scale, dtype=torch.float32),
        )

        return data_item
    

class TrajectoryDatasetTest(Dataset):
    def __init__(self, data, scale=10.0):
        """
        data: Shape (N, 50, 110, 6) Testing data
        scale: Scale for normalization (suggested to use 10.0 for Argoverse 2 data)
        """
        self.data = data
        self.scale = scale

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Testing data only contains historical trajectory
        scene = self.data[idx]  # (50, 50, 6)
        hist = scene.copy()
        
        origin = hist[0, 49, :2].copy()
        hist[..., :2] = hist[..., :2] - origin
        hist[..., :4] = hist[..., :4] / self.scale

        data_item = Data(
            x=torch.tensor(hist, dtype=torch.float32),
            origin=torch.tensor(origin, dtype=torch.float32).unsqueeze(0),
            scale=torch.tensor(self.scale, dtype=torch.float32),
        )
        return data_item

In [5]:
torch.manual_seed(251)
np.random.seed(42)

scale = 7.0

N = len(train_data)
val_size = int(0.1 * N)
train_size = N - val_size

train_dataset = TrajectoryDatasetTrain(train_data[:train_size], scale=scale, augment=True)
val_dataset = TrajectoryDatasetTrain(train_data[train_size:], scale=scale, augment=False)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: Batch.from_data_list(x))

# Set device for training speedup
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Using Apple Silicon GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA GPU")
else:
    device = torch.device('cpu')

Using CUDA GPU


# Use Transformer to Reconstruct

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import numpy as np
import pandas as pd
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader

In [27]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=100):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (B, T, D)
        return x + self.pe[:, :x.size(1), :]

class TransformerModel(nn.Module):
    def __init__(self, input_dim=6, d_model=512, nhead=16, num_layers=4, 
                 output_dim=60 * 2, seq_len=50):
        super(TransformerModel, self).__init__()
        self.seq_len = seq_len
        
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=d_model*4,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.predict_head = nn.Sequential(
            nn.Linear(d_model * seq_len, d_model*2),
            nn.ReLU(),
            nn.Linear(d_model*2, output_dim)
        )

    def forward(self, data):
        x = data.x.view(-1, 50, 50, 6)[:, 0, :, :]  # (B, T, F)
        B, T, F = x.shape
        
        x = self.input_proj(x)  # (B, T, d_model)
        x = self.pos_encoder(x)
        
        transformer_out = self.transformer(x)  # (B, T, d_model)

        flattened = transformer_out.reshape(B, -1)  # (B, T * d_model)
        
        out = self.predict_head(flattened)  # (B, 120)
        return out.view(-1, 60, 2)  # (B, 60, 2)

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TransformerModel(d_model=512, nhead=64, num_layers=4).to(device)


optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)


scheduler = optim.lr_scheduler.SequentialLR(
    optimizer,
    schedulers=[
        optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, total_iters=10),
        optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40, eta_min=1e-6)
    ],
    milestones=[10]
)


criterion = nn.SmoothL1Loss(beta=0.5)

early_stopping_patience = 30
best_val_loss = float('inf')
no_improvement = 0

In [40]:
for epoch in tqdm.tqdm(range(100), desc="Epoch", unit="epoch"):
    # ---- Training ----
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        batch = batch.to(device)
        pred = model(batch)
        y = batch.y.view(batch.num_graphs, 60, 2)
        loss = criterion(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
    
    # ---- Validation ----
    model.eval()
    val_loss = 0
    val_mae = 0
    val_mse = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = batch.to(device)
            pred = model(batch)
            y = batch.y.view(batch.num_graphs, 60, 2)
            val_loss += criterion(pred, y).item()

            pred_denorm = pred * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            y_denorm = y * batch.scale.view(-1, 1, 1) + batch.origin.unsqueeze(1)
            
            val_mae += nn.L1Loss()(pred_denorm, y_denorm).item()
            val_mse += nn.MSELoss()(pred_denorm, y_denorm).item()
    
    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_mae /= len(val_dataloader)
    val_mse /= len(val_dataloader)
    
    scheduler.step()
    
    tqdm.tqdm.write(f"Epoch {epoch:03d} | LR {optimizer.param_groups[0]['lr']:.6f} | "
                   f"Train Loss {train_loss:.4f} | Val Loss {val_loss:.4f} | "
                   f"Val MAE {val_mae:.4f} | Val MSE {val_mse:.4f}")
    
    if val_loss < best_val_loss - 1e-4:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        no_improvement += 1
        if no_improvement >= early_stopping_patience:
            print("Early stopping triggered!")
            break

Epoch:   0%|                                                                                | 0/100 [00:00<?, ?epoch/s]C:\Users\dysr3\AppData\Local\Temp\ipykernel_11284\3713195397.py:30: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future @ R
C:\Users\dysr3\AppData\Local\Temp\ipykernel_11284\3713195397.py:39: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  future = future - origin
Epoch:   1%|▋                                                                       | 1/100 [00:09<15:06,  9.16s/epoch]

Epoch 000 | LR 0.000033 | Train Loss 0.6456 | Val Loss 0.4200 | Val MAE 4.0030 | Val MSE 51.1283


Epoch:   2%|█▍                                                                      | 2/100 [00:18<15:02,  9.21s/epoch]

Epoch 001 | LR 0.000062 | Train Loss 0.3597 | Val Loss 0.2927 | Val MAE 3.1312 | Val MSE 29.2511


Epoch:   3%|██▏                                                                     | 3/100 [00:27<14:50,  9.18s/epoch]

...

Epoch 080 | LR 0.000264 | Train Loss 0.1261 | Val Loss 0.1241 | Val MAE 1.6682 | Val MSE 9.6419


Epoch:  82%|██████████████████████████████████████████████████████████▏            | 82/100 [12:28<02:47,  9.28s/epoch]

Epoch 081 | LR 0.000271 | Train Loss 0.1268 | Val Loss 0.1402 | Val MAE 1.8702 | Val MSE 11.2679


Epoch:  82%|██████████████████████████████████████████████████████████▏            | 82/100 [12:37<02:46,  9.24s/epoch]

Epoch 082 | LR 0.000278 | Train Loss 0.1256 | Val Loss 0.1277 | Val MAE 1.7105 | Val MSE 10.4434
Early stopping triggered!


In [43]:
test_dataset = TrajectoryDatasetTest(test_data, scale=scale)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                         collate_fn=lambda xs: Batch.from_data_list(xs))

best_model = torch.load("best_model.pt")
model = TransformerModel(d_model=512, nhead=64, num_layers=4).to(device)
model.load_state_dict(best_model)
model.eval()

pred_list = []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        pred_norm = model(batch)
        
        pred = pred_norm * batch.scale.view(-1,1,1) + batch.origin.unsqueeze(1)
        pred_list.append(pred.cpu().numpy())

pred_list = np.concatenate(pred_list, axis=0)
pred_output = pred_list.reshape(-1, 2)
output_df = pd.DataFrame(pred_output, columns=['x', 'y'])
output_df.index.name = 'index'
output_df.to_csv('submission.csv', index=True)